In [45]:
# define hosts for which model should be built
hosts = ["hosttaxID1", "hosttaxID2"]

In [46]:
# create a untrained classifier for each host
from classification.ElkanotoSVCClassifier import ElkanotoSVCClassifier

classifiers = {}
for host in hosts:
    classifiers[host] = ElkanotoSVCClassifier()

In [47]:
# load datasets
import numpy as np
import pandas as pd
import ast

# read in file
featureFilePath = "mock_features_preprocessing.csv"
featureData = pd.read_csv(featureFilePath)


# Convert the flat feature list into right format for classifier
featureData['FeaturesFlat'] = featureData['FeaturesFlat'].apply(ast.literal_eval)
featureData['FeaturesFlat'] = featureData['FeaturesFlat'].apply(lambda x: [float(i) for i in x])


# Create dictionary of hostIDs and empty lists (of feature lists for each datapoint) for each host
featuresPerHost = {}
for host in hosts:
    featuresPerHost[host] = []


# Extract features of virus sequences for each host in seperate list and convert them to a np array
for _, row in featureData.iterrows():
    host = row['HostTaxID']
    features = np.array(row['FeaturesFlat'])
    featuresPerHost[host].append(features)

for host in hosts:
    featuresPerHost[host] = np.array(featuresPerHost[host])


In [48]:
# transform features into input data
X = np.concatenate([featuresPerHost[host] for host in hosts])

def getYForHost(h):
    y = []
    for host in hosts:
        y += [1 if host == h else 0] * len(featuresPerHost[host])

    return np.array(y)

In [49]:
# run k-fold cross validation for each host
from sklearn.model_selection import KFold, cross_validate
import numpy as np

scores = {}
for host in hosts:
    # load the training data
    X = X
    y = getYForHost(host)

    # run k-fold cross validation
    cv = KFold(n_splits=5, shuffle=True)
    scores[host] = cross_validate(classifiers[host], X, y, cv=cv, scoring=['balanced_accuracy', 'recall', 'precision'])

/Users/al/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/Users/al/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/al/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/al/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, 

In [50]:
# print out the metrics we want to see using the scores
scores

{'hosttaxID1': {'fit_time': array([0.00468397, 0.00295091, 0.00166297, 0.00072789, 0.00107598]),
  'score_time': array([0.00875211, 0.00752616, 0.00493312, 0.        , 0.        ]),
  'test_balanced_accuracy': array([ 1.,  0.,  1., nan, nan]),
  'test_recall': array([ 1.,  0.,  1., nan, nan]),
  'test_precision': array([ 1.,  0.,  1., nan, nan])},
 'hosttaxID2': {'fit_time': array([0.00170922, 0.00112796, 0.0010488 , 0.0011251 , 0.0008769 ]),
  'score_time': array([0.        , 0.        , 0.00421023, 0.        , 0.        ]),
  'test_balanced_accuracy': array([nan, nan,  0., nan, nan]),
  'test_recall': array([nan, nan,  0., nan, nan]),
  'test_precision': array([nan, nan,  0., nan, nan])}}